In [6]:
import pandas as pd
import matplotlib as plt
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
import requests
from datetime import datetime, timedelta

In [7]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm  

all_protocols = requests.get("https://api.llama.fi/protocols").json()

eth_protocols = [p for p in all_protocols if "Ethereum" in p.get("chains", [])]

print(f"✅ Number of all protocols on Ethereum: {len(eth_protocols)}")

one_year_ago = datetime.now() - timedelta(days=365)

all_data = []

for proto in tqdm(eth_protocols):
    name = proto["slug"]  # مثل 'aave'
    try:
        url = f"https://api.llama.fi/protocol/{name}"
        res = requests.get(url)
        if res.status_code != 200:
            continue

        data = res.json()
        tvl_data = data.get("chainTvls", {}).get("Ethereum", {}).get("tvl", [])
        if not tvl_data:
            continue

        df = pd.DataFrame(tvl_data)
        df['date'] = pd.to_datetime(df['date'], unit='s')
        df = df[df['date'] >= one_year_ago]
        df['tvl_usd'] = df['totalLiquidityUSD']
        df['protocol_name'] = name
        df['chain'] = "Ethereum"
        df = df[['date', 'tvl_usd', 'protocol_name', 'chain']]
        
        all_data.append(df)

    except Exception as e:
        print(f"❌ problem in {name}: {e}")
        continue

# 6. نهایی کردن دیتافریم
final_df = pd.concat(all_data).reset_index(drop=True)
print(f"\n✅ Done! {len(final_df)} number of extracted rows.")
print(final_df.head())

✅ تعداد پروتکل‌های روی Ethereum: 1453


100%|███████████████████████████████████████| 1453/1453 [15:20<00:00,  1.58it/s]


✅ تمام شد! 446271 ردیف داده استخراج شد.
        date       tvl_usd protocol_name     chain
0 2024-04-14  2.650700e+10   binance-cex  Ethereum
1 2024-04-15  2.792865e+10   binance-cex  Ethereum
2 2024-04-16  2.790882e+10   binance-cex  Ethereum
3 2024-04-17  2.770807e+10   binance-cex  Ethereum
4 2024-04-18  2.711308e+10   binance-cex  Ethereum


In [8]:
final_df.head()
print(final_df.shape)

(446271, 4)


In [9]:
final_df.head()

,date,tvl_usd,protocol_name,chain
0,2024-04-14,2.650700e+10,binance-cex,Ethereum
1,2024-04-15,2.792865e+10,binance-cex,Ethereum
2,2024-04-16,2.790882e+10,binance-cex,Ethereum
3,2024-04-17,2.770807e+10,binance-cex,Ethereum
4,2024-04-18,2.711308e+10,binance-cex,Ethereum
